In [1]:
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
#import matplotlib.pyplot as plt
# from pyscf.geomopt.berny_solver import optimize
from pyscf.grad import rhf as grhf
#from pyscf.hessian import rhf as hrhf
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
angstrom = 1 / 0.52917721067
from pyscf.scf._response_functions import _gen_rhf_response 
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV.reshape((1,dV.shape[0],dV.shape[1]))

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

In [2]:
import time
from functools import reduce
import numpy
from pyscf import lib
from pyscf.lib import logger
from pyscf.scf import cphf
from pyscf.prop.nmr import rhf as rhf_nmr

def polarizability(polobj, with_cphf=True):
    mf = polobj._scf
    mol = mf.mol
    mo_energy = mf.mo_energy
    mo_coeff = mf.mo_coeff
    mo_occ = mf.mo_occ
    occidx = mo_occ > 0
    orbo = mo_coeff[:, occidx]
    orbv = mo_coeff[:,~occidx]

    charges = mol.atom_charges()
    coords  = mol.atom_coords()
    charge_center = numpy.einsum('i,ix->x', charges, coords) / charges.sum()
    int_r=DeltaV(mol,[.0001,-.0001])    ########   .001 as finite difference intervall  
    h1 = lib.einsum('xpq,pi,qj->xij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals?
    print(h1.shape, "shape (1, n ao, nocc)" )
    #print(mo_energy, mo_occ,h1)
    s1 = numpy.zeros_like(h1)
    vind = polobj.gen_vind(mf, mo_coeff, mo_occ)
    if with_cphf:
        mo1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1, polobj.max_cycle_cphf, polobj.conv_tol)[0]
    else:
        mo1 = rhf_nmr._solve_mo1_uncoupled(mo_energy, mo_occ, h1, s1)[0]
    #e2 = numpy.einsum('xpi,ypi->xy', h1, mo1)
    # *-1 from the definition of dipole moment. *2 for double occupancy
    #e2 = (e2 + e2.T) * -2
    #return e2
    return mo1

In [3]:
mol = gto.M(atom='C 0 0 0; O 0 0 2.0', unit="Bohr",basis="sto-3g")
mf = scf.RHF(mol)
e=mf.scf()

converged SCF energy = -111.199724042754


In [4]:
g = mf.Gradients()
g.kernel()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.3426279915
1 O    -0.0000000000     0.0000000000    -0.3426279915
----------------------------------------------


array([[ 2.50345005e-16, -2.80542611e-16,  3.42627991e-01],
       [-2.50345005e-16,  2.80542611e-16, -3.42627991e-01]])

In [5]:
mol.atom_charge(0)

6

In [6]:
mf1=fc(mf,[.0001,-.0001])
mf1.scf()

converged SCF energy = -111.198995727549


/home/administrator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


-111.19899572754903

In [7]:
C1=mf1.mo_coeff


In [8]:
mf2=fc(mf,[.0001,-.0001])
mf2.scf()

converged SCF energy = -111.198995727549


/home/administrator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


-111.19899572754903

In [9]:
e=mf.mo_energy
e1=mf1.mo_energy
e2=mf2.mo_energy

In [10]:
e,e1,e2

(array([-20.46076589, -11.10193184,  -1.52172836,  -0.71090808,
         -0.5989603 ,  -0.5989603 ,  -0.45135035,   0.34504015,
          0.34504015,   1.17302526]),
 array([-20.46017744, -11.10237896,  -1.52171628,  -0.71090629,
         -0.59895416,  -0.59895416,  -0.45136514,   0.34503314,
          0.34503314,   1.17302357]),
 array([-20.46017744, -11.10237896,  -1.52171628,  -0.71090629,
         -0.59895416,  -0.59895416,  -0.45136514,   0.34503314,
          0.34503314,   1.17302357]))

In [11]:
(e1-e)/.001,(e2-e)/.0001

(array([ 0.58844667, -0.44711917,  0.01208183,  0.00179161,  0.00614406,
         0.00614406, -0.01479785, -0.00701284, -0.00701284, -0.00169259]),
 array([ 5.88446669, -4.47119171,  0.12081825,  0.0179161 ,  0.0614406 ,
         0.0614406 , -0.14797845, -0.07012837, -0.07012837, -0.01692594]))

## Obtaining derivatives of W
$$W=  \sum_i ^{mo.occ.} \epsilon_i C_{\mu i} C_{\nu i}^\dagger 
$$

$$ \frac{\partial W}{\partial Z_I}= \sum_i ^{mo.occ.} \left( \epsilon_i (CU)_{\mu i} C_{\nu i}^\dagger + 
\epsilon_i C_{\mu i} (CU)^\dagger_{\nu i}   +\frac{\partial \epsilon_i}{\partial Z_I} C_{\mu i} C_{\nu i}^\dagger \right)$$

In [12]:
g1=mf1.Gradients()

In [13]:
W1=g1.make_rdm1e()
W=g.make_rdm1e()
dW=W1-W

In [14]:
o=mf.mo_occ
e=mf.mo_energy
C=mf.mo_coeff
S=mf.get_ovlp()
np.allclose(C@np.diag(o*e)@C.T,g.make_rdm1e())

True

In [15]:
plo=mf.Polarizability()
pa=polarizability(plo)
nao=mol.nao
nocc=mol.nelectron//2 #RHF
U=np.zeros((nao,nao))
U[:,:nocc]=pa[0,:,:nocc]
U=U.T-U
O=np.diag(mf.mo_occ)
dP_app=C@(U@O-O@U)@C.T

(1, 10, 7) shape (1, n ao, nocc)


In [16]:
np.allclose(C1@np.diag(o*e1)@C1.T,g1.make_rdm1e())

True

In [17]:
print(C1-C),print(C@U) #are different ! Orbital Mixing !!!! 

[[ 1.40205359e-07  2.31233600e-06 -2.72598827e-06  4.07647120e-07
  -2.13179024e-17  4.31853803e-16  1.26063071e-05  2.52717002e-17
   3.15600175e-16  4.78066596e-06]
 [-3.24642228e-06 -9.69584425e-06  3.43116823e-05 -3.51273010e-05
   4.84972394e-16 -1.37695219e-15 -2.71781175e-05  1.20306580e-16
  -2.21430501e-15 -1.20774775e-06]
 [-1.08957851e-16  8.98556133e-19 -8.72664263e-17 -5.35966501e-16
   1.25383939e-04  4.03004510e-02 -2.17861293e-16  3.18272948e-04
  -2.33245864e-02  1.34392885e-16]
 [-4.50946495e-16  1.75136845e-18  2.35480287e-16 -2.15691800e-15
  -4.03004510e-02  1.25383939e-04 -4.30422476e-16  2.33245864e-02
   3.18272948e-04  1.69134328e-15]
 [-2.71612381e-06 -2.29727940e-06  1.57660300e-05  1.80328266e-05
   5.06265669e-17 -1.91200893e-15 -1.26890885e-05  2.96041965e-16
  -1.65589831e-15 -6.33976081e-06]
 [-1.66748285e-06 -1.12028994e-07  6.79405399e-07 -9.03341683e-06
  -1.97766971e-18 -4.59381088e-16 -3.92214704e-06  6.89359421e-17
  -1.87139875e-17  4.90498150e-06

(None, None)

In [18]:
dC=C@U

In [19]:
print(np.linalg.norm((C+dC)@np.diag(o*e1)@(C+dC).T-W1),np.linalg.norm(W1))
print(np.linalg.norm((C1)@np.diag(o*e1)@(C1).T-W1))

0.0011438938703716604 46.338602110654726
0.0


In [20]:
print(np.linalg.norm(dW-C@np.diag(o*(e1 -e))@C.T-dC@np.diag(o*e)@C.T-C@np.diag(o*e)@(dC).T))
print(np.linalg.norm(dW))

0.0011438977659259995
0.001747326543856332


In [21]:
print(np.linalg.norm((C+dC)@np.diag(o*e1)@(C+dC).T-W),np.linalg.norm(dW))

0.0015202689475889118 0.001747326543856332


In [22]:
F=mf.get_fock()

In [23]:
F1=mf1.get_fock()
print(mf1.mo_energy)
np.sort(np.linalg.eig(np.linalg.inv(S)@F1)[0])

[-20.46017744 -11.10237896  -1.52171628  -0.71090629  -0.59895416
  -0.59895416  -0.45136514   0.34503314   0.34503314   1.17302357]


array([-20.46017704+0.00000000e+00j, -11.1023791 +0.00000000e+00j,
        -1.52171615+0.00000000e+00j,  -0.71090614+0.00000000e+00j,
        -0.59895404+0.00000000e+00j,  -0.59895404+0.00000000e+00j,
        -0.4513652 +0.00000000e+00j,   0.34503314-1.34143225e-16j,
         0.34503314+1.34143225e-16j,   1.17302361+0.00000000e+00j])

In [24]:
S1=mf1.get_ovlp()
np.allclose(S,S1)
dV=DeltaV(mol,[.0001,-.0001])

In [25]:
np.linalg.norm(F+dV-F1)

0.0001701633132641137

In [26]:
np.linalg.norm(dV)

0.000899137263275962

In [27]:
np.linalg.norm(F-F1)

0.0007564728019632337

In [28]:
g_ijkl=mf.mol.intor('int2e_sph')

In [29]:
dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP_app)*2-np.einsum('ijkl,jl->ik',g_ijkl,dP_app)

In [30]:
np.linalg.norm(F1-F-dV+dF2el/2)

1.8888237620809944e-09

In [31]:
np.allclose(np.sort(np.linalg.eig(np.linalg.inv(S)@(F+dV-dF2el/2))[0]),e1)

True

In [32]:
#dP_app=C@(U@O-O@U)@C.T
#dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP_app)*2-np.einsum('ijkl,jl->ik',g_ijkl,dP_app)
#e1_app=np.sort(np.linalg.eig(np.linalg.inv(S)@(F+dV-dF2el/2))